In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from crazyflie_description_py.parameters import MIC_POSITIONS, FS
import pyroomacoustics as pra
from audio_stack.beam_former import rotate_mics
from signals import generate_signal

def generate_room(distance_cm=0, yaw_deg=0, ax=None):
    frequency_hz = 4125
    room_dim = np.array([10, 7])
    yaw_offset = -120
    x_offset = 5
    y_offset = 0.1
    room = pra.ShoeBox(fs=FS, p=room_dim)

    offset = [x_offset, y_offset + distance_cm * 1e-2]
    mic_positions = np.array(MIC_POSITIONS)
    mic_positions = offset + rotate_mics(mic_positions, yaw_offset - yaw_deg)

    buzzer_signal = generate_signal(signal_type='mono', Fs=FS, duration_sec=30, frequency_hz=frequency_hz, max_dB=-10)
    room.add_source(offset, signal=buzzer_signal)

    beam_former = pra.Beamformer(mic_positions.T,  room.fs)
    room.add_microphone_array(beam_former)
    
    if ax is not None:
        for i, mic in enumerate(mic_positions):
            ax.scatter(*mic, label=f'mic{i}')
        ax.plot([0, room_dim[0]], [0, 0], label='wall')
        ax.axis('equal')
        ax.set_xlim(4.5, 5.5)
        ax.set_ylim(-0.1, 1.0)
        ax.scatter(*offset, label='buzzer')
        ax.legend()

    room.simulate()
    return room

fig, axs = plt.subplots(1, 2)
fig.set_size_inches(15, 5)
room = generate_room(distance_cm=20, yaw_deg=0, ax=axs[0])
axs[0].set_title(0)
room = generate_room(distance_cm=50, yaw_deg=27, ax=axs[1])
axs[1].set_title(27)

In [ ]:
N_BUFFER = 2**10
signals_f = np.array(
    [
        pra.transform.stft.analysis(signal, N_BUFFER, N_BUFFER // 2).T
        for signal in room.mic_array.signals
    ]
).transpose(2, 0, 1) # n_mics x n_frequencies x n_times
frequencies = np.arange(N_BUFFER // 2 + 1) * room.fs / N_BUFFER

fig, axs = plt.subplots(signals_f.shape[1], 1, sharex=True)
fig.set_size_inches(8, 2.5 * signals_f.shape[1])
for i in range(signals_f.shape[1]):
    axs[i].pcolormesh(range(signals_f.shape[0]), frequencies, np.log10(np.abs(signals_f[:, i].T)))
    axs[i].set_ylabel("frequency [Hz]")
    axs[i].set_title(f"mic{i}")
axs[i].set_xlabel("time index")

In [ ]:
df_total = pd.DataFrame(columns=['signals_f', 'degree', 'yaw', 'distance', 'source', 'psd', 'spec', 'frequencies'])

degree = 0
source = 'mono4125'

for distance_cm in np.arange(100, step=10):
    room = generate_room(distance_cm=distance_cm, yaw_deg=degree)
        
    N_BUFFER = 2**10
    signals_f = np.array(
        [
            pra.transform.stft.analysis(signal, N_BUFFER, N_BUFFER // 2).T
            for signal in room.mic_array.signals
        ] # n_mics x n_freqs x n_times
    ).transpose(2, 0, 1) # n_times x n_mics x n_freqs
    frequencies = np.arange(N_BUFFER // 2 + 1) * room.fs / N_BUFFER
    
    yaw = np.full(signals_f.shape[0], -degree)
    
    spec = np.sum(np.abs(signals_f), axis=1)
    
    # TODO(FD): implement if needed. 
    psd = None 
    
    df_total.loc[len(df_total), :] = dict(
        degree=degree,
        yaw=yaw,
        distance=distance_cm,
        source=str(source),
        signals_f=signals_f,
        frequencies=frequencies,
        spec=spec,
        psd=psd
    ) 